In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import display

## **Main Specifications + Making the Figures**

### Figure 1: Coefficient Plot for the Main DiD

1. No controls or FEs
2. Controls but no FEs
3. FEs but no controls
4. Both controls and FEs

### Figure 2: 2-Panel Coefficient Plots

1. Low income groups
2. Poverty vs non-poverty tows

### **Figure 1**

In [ ]:
# "raw formula" - no controls, FEs
formula_1 = """
    redeemed ~ Post_Waiver2020 * treat_lowinc_proxy
               + Post_Waiver2021 * treat_lowinc_proxy
"""
model_1 = smf.ols(formula=formula_1, data=tow_2018_25).fit(cov_type='HC1')

# variables of interest
results_1 = pd.DataFrame({
    "coef": model_1.params,
    "se": model_1.bse,
    "pval": model_1.pvalues
    })

# keep only treatment/interactions
( results_1.loc[results_1.index.str
                .contains("Post_Waiver2020|Post_Waiver2021|treat_lowinc_proxy")]
)

coef        se           pval
Post_Waiver2020                     0.016261  0.001676   2.934404e-22
treat_lowinc_proxy                 -0.234664  0.006547  2.198092e-281
Post_Waiver2020:treat_lowinc_proxy  0.038597  0.003969   2.379736e-22
Post_Waiver2021                     0.009079  0.003115   3.564937e-03
Post_Waiver2021:treat_lowinc_proxy -0.004956  0.007332   4.991023e-01

In [ ]:
# controls, but no FEs
formula_2 = """
    redeemed ~ Post_Waiver2020 * treat_lowinc_proxy
               + Post_Waiver2021 * treat_lowinc_proxy
               + poverty_tow + repeat_tow + out_of_state
"""
model_2 = smf.ols(formula=formula_2, data=tow_2018_25).fit(cov_type='HC1')

# variables of interest
results_2 = pd.DataFrame({
    "coef": model_2.params,
    "se": model_2.bse,
    "pval": model_2.pvalues
    })

# keep only treatment/interactions
( results_2.loc[results_2.index.str
                .contains("Post_Waiver2020|Post_Waiver2021|treat_lowinc_proxy")]
)

coef        se           pval
Post_Waiver2020                     0.017853  0.001642   1.584126e-27
treat_lowinc_proxy                 -0.179539  0.005966  5.571928e-199
Post_Waiver2020:treat_lowinc_proxy  0.031129  0.003648   1.421914e-17
Post_Waiver2021                     0.003544  0.003027   2.416621e-01
Post_Waiver2021:treat_lowinc_proxy  0.003658  0.006661   5.829444e-01

In [ ]:
# FEs, but no controls
formula_3 = """
    redeemed ~ Post_Waiver2020 * treat_lowinc_proxy
               + Post_Waiver2021 * treat_lowinc_proxy
               + C(month_year_cat) + C(day_of_month_cat)
"""
model_3 = smf.ols(formula=formula_3, data=tow_2018_25).fit(cov_type='HC1')

# variables of interest
results_3 = pd.DataFrame({
    "coef": model_3.params,
    "se": model_3.bse,
    "pval": model_3.pvalues
    })

# keep only treatment/interactions
( results_3.loc[results_3.index.str
                .contains("Post_Waiver2020|Post_Waiver2021|treat_lowinc_proxy")]
)

coef            se           pval
Post_Waiver2020                    -2.067716e+09  9.646296e+08   3.207026e-02
treat_lowinc_proxy                 -2.342519e-01  6.544092e-03  1.277445e-280
Post_Waiver2020:treat_lowinc_proxy  4.462178e-02  3.968306e-03   2.463764e-29
Post_Waiver2021                    -8.415763e-03  1.279688e-02   5.107684e-01
Post_Waiver2021:treat_lowinc_proxy -4.663091e-03  7.332751e-03   5.248243e-01

In [ ]:
# "fully saturated" - controls and FEs
formula_4 = """
    redeemed ~ Post_Waiver2020 * treat_lowinc_proxy
               + Post_Waiver2021 * treat_lowinc_proxy
               + poverty_tow + repeat_tow + out_of_state
               + C(month_year_cat) + C(day_of_month_cat)
"""
model_4 = smf.ols(formula=formula_4, data=tow_2018_25).fit(cov_type='HC1')

# variables of interest
results_4 = pd.DataFrame({
    "coef": model_4.params,
    "se": model_4.bse,
    "pval": model_4.pvalues
    })

# keep only treatment/interactions
( results_4.loc[results_4.index.str
                .contains("Post_Waiver2020|Post_Waiver2021|treat_lowinc_proxy")]
)

coef            se           pval
Post_Waiver2020                    -1.989983e+09  9.732679e+08   4.089027e-02
treat_lowinc_proxy                 -1.789372e-01  5.964985e-03  1.044310e-197
Post_Waiver2020:treat_lowinc_proxy  3.535395e-02  3.641833e-03   2.794772e-22
Post_Waiver2021                    -8.897210e-03  1.219619e-02   4.656915e-01
Post_Waiver2021:treat_lowinc_proxy  4.789097e-03  6.661509e-03   4.721898e-01

In [ ]:
# prepare the table for plotting
formulas = [formula_1, formula_2, formula_3, formula_4]
labels   = ( ["No FEs / No controls", "Controls only", 
              "FEs only", "FEs + controls"]
)

results_list = []

for f, label in zip(formulas, labels):
    model = smf.ols(formula=f, data=tow_2018_25).fit(cov_type="HC1")
    coef = model.params["Post_Waiver2020:treat_lowinc_proxy"]
    se   = model.bse["Post_Waiver2020:treat_lowinc_proxy"]
    results_list.append({"spec": label, "coef": coef, "se": se})

results_df = pd.DataFrame(results_list)
results_df["ci_low"] = results_df["coef"] - 1.96 * results_df["se"]
results_df["ci_high"] = results_df["coef"] + 1.96 * results_df["se"]

In [103]:
plt.figure(figsize=(8, 5))

# --- Error bars (drawn first so they’re behind points) ---
plt.errorbar(
    results_df["coef"],
    results_df["spec"],   # or "model" if that's your column name
    xerr=[results_df["coef"] - results_df["ci_low"], 
          results_df["ci_high"] - results_df["coef"]],
    fmt="none",
    color="gray",
    capsize=5,
    zorder=1
)

# --- Scatter points with seaborn colors ---
ax = sns.scatterplot(
    data=results_df,
    x="coef", y="spec",
    hue="spec",           # one color per model/spec
    palette="Set2",
    s=80,
    legend=False,
    zorder=2
)

# --- Annotate coefficient values ---
for i, row in results_df.iterrows():
    plt.text(
        row["coef"], i + 0.15,
        f'{row["coef"]:.3f}',
        ha="center", color="black"
    )

# --- Aesthetics ---
ax.axvline(0, color="black", linestyle="--", zorder=0)
ax.grid(True, linestyle="--", alpha=0.3)

plt.title("DiD Estimates of the 2020 Tow Fee Waiver Effect Across Models")
plt.xlabel("Coefficient Estimate")
plt.ylabel("Model")
plt.ylim(-0.5, len(results_df) - 0.5)
plt.tight_layout()

plt.savefig("figure_1.png", dpi=300, bbox_inches="tight")

plt.show()

### **Figure 2**

In [ ]:
# triple diff - fully saturated
formula_ddd_age_fullsat = """
    redeemed ~ Post_Waiver2020 * (lowinc_10_to_15 + lowinc_16_to_20 + lowinc_over_20)
               + Post_Waiver2021 * (lowinc_10_to_15 + lowinc_16_to_20 + lowinc_over_20)
               + poverty_tow + repeat_tow + out_of_state
               + C(month_year_cat) + C(day_of_month_cat)
"""

model_ddd_age_fullsat = smf.ols(formula=formula_ddd_age_fullsat, data=tow_2018_25).fit(cov_type='HC1')

# variables of interest
results_ddd_age_fullsat = pd.DataFrame({
    "coef": model_ddd_age_fullsat.params,
    "se": model_ddd_age_fullsat.bse,
    "pval": model_ddd_age_fullsat.pvalues
})

# keep only treatment/interactions
( results_ddd_age_fullsat.loc[results_ddd_age_fullsat.index.str
            .contains("Post_Waiver2020|Post_Waiver2021|lowinc_10_to_15|lowinc_16_to_20|lowinc_over_20")]
)

coef            se           pval
Post_Waiver2020                 -4.326447e+08  2.931248e+08   1.399508e-01
lowinc_10_to_15                 -4.981247e-02  7.258074e-03   6.740920e-12
lowinc_16_to_20                 -2.498688e-01  9.881004e-03  4.351492e-141
lowinc_over_20                  -3.723216e-01  1.232977e-02  2.596757e-200
Post_Waiver2020:lowinc_10_to_15  2.862405e-02  4.611659e-03   5.404436e-10
Post_Waiver2020:lowinc_16_to_20  7.343473e-02  6.180737e-03   1.481764e-32
Post_Waiver2020:lowinc_over_20   5.788603e-02  6.970014e-03   9.981275e-17
Post_Waiver2021                 -1.244634e-02  1.202130e-02   3.005020e-01
Post_Waiver2021:lowinc_10_to_15 -8.666861e-03  8.203303e-03   2.907359e-01
Post_Waiver2021:lowinc_16_to_20  7.055905e-03  1.108233e-02   5.243330e-01
Post_Waiver2021:lowinc_over_20   5.662909e-02  1.363145e-02   3.262890e-05

In [ ]:
# DiD - poverty tows - fully saturated

formula_did_pov_fullsat = """
    redeemed ~ Post_Waiver2020 * treat_lowinc_proxy
               + Post_Waiver2021 * treat_lowinc_proxy
               + poverty_tow + repeat_tow + out_of_state
               + C(month_year_cat) + C(day_of_month_cat)
"""

model_did_pov_fullsat = smf.ols(formula=formula_did_pov_fullsat, data=tow_2018_25_pov).fit(cov_type='HC1')

# variables of interest
results_did_pov_fullsat = pd.DataFrame({
    "coef": model_did_pov_fullsat.params,
    "se": model_did_pov_fullsat.bse,
    "pval": model_did_pov_fullsat.pvalues
})

# keep only treatment/interactions
( results_did_pov_fullsat.loc[results_did_pov_fullsat.index.str
                              .contains("Post_Waiver2020|Post_Waiver2021|treat_lowinc_proxy")]
)

coef            se          pval
Post_Waiver2020                     2.568976e+10  1.801483e+10  1.538585e-01
treat_lowinc_proxy                 -3.106503e-01  1.520001e-02  7.760662e-93
Post_Waiver2020:treat_lowinc_proxy  6.087187e-02  9.668355e-03  3.054428e-10
Post_Waiver2021                     5.205840e-02  4.097986e-02  2.039631e-01
Post_Waiver2021:treat_lowinc_proxy -1.785994e-02  1.710380e-02  2.963888e-01

In [ ]:
# DiD - non-poverty tows - fully saturated

formula_did_nonpov_fullsat = """
    redeemed ~ Post_Waiver2020 * treat_lowinc_proxy
               + Post_Waiver2021 * treat_lowinc_proxy
               + poverty_tow + repeat_tow + out_of_state
               + C(month_year_cat) + C(day_of_month_cat)
"""

model_did_nonpov_fullsat = smf.ols(formula=formula_did_nonpov_fullsat, data=tow_2018_25_nonpov).fit(cov_type='HC1')

# variables of interest
results_did_nonpov_fullsat = pd.DataFrame({
    "coef": model_did_nonpov_fullsat.params,
    "se": model_did_nonpov_fullsat.bse,
    "pval": model_did_nonpov_fullsat.pvalues
})

# keep only treatment/interactions
( results_did_nonpov_fullsat.loc[results_did_nonpov_fullsat.index.str
        .contains("Post_Waiver2020|Post_Waiver2021|treat_lowinc_proxy")]
)

coef            se          pval
Post_Waiver2020                    -1.639090e+09  8.066552e+08  4.215780e-02
treat_lowinc_proxy                 -1.226894e-01  6.638396e-03  2.894758e-76
Post_Waiver2020:treat_lowinc_proxy  9.489692e-03  4.011821e-03  1.800901e-02
Post_Waiver2021                    -1.586073e-02  1.229392e-02  1.970063e-01
Post_Waiver2021:treat_lowinc_proxy  3.980243e-03  7.436104e-03  5.924706e-01

In [ ]:
# preparing the table

# --- Extract Triple Diff ---
vars_triple = [
    "Post_Waiver2020:lowinc_10_to_15",
    "Post_Waiver2020:lowinc_16_to_20",
    "Post_Waiver2020:lowinc_over_20"
]
triple_df = results_ddd_age_fullsat.loc[results_ddd_age_fullsat.index.isin(vars_triple)].copy()
triple_df = triple_df.reset_index().rename(columns={"index": "variable"})
triple_df["group"] = triple_df["variable"].replace({
    "Post_Waiver2020:lowinc_10_to_15": "Age 10–15",
    "Post_Waiver2020:lowinc_16_to_20": "Age 16–20",
    "Post_Waiver2020:lowinc_over_20": "Age >20"
})
triple_df["model"] = "Triple Diff (by Age)"

# --- Extract DiD: Poverty ---
pov_var = "Post_Waiver2020:treat_lowinc_proxy"
pov_df = results_did_pov_fullsat.loc[[pov_var]].copy().reset_index().rename(columns={"index": "variable"})
pov_df["group"] = "Low-income (poverty tows)"
pov_df["model"] = "DiD (poverty tows)"

# --- Extract DiD: Non-poverty ---
nonpov_df = ( results_did_nonpov_fullsat.loc[[pov_var]].copy().reset_index()
             .rename(columns={"index": "variable"})
)
nonpov_df["group"] = "Low-income (non-poverty tows)"
nonpov_df["model"] = "DiD (non-poverty tows)"

# --- Combine all ---
coef_df = pd.concat([triple_df, pov_df, nonpov_df], ignore_index=True)
coef_df["ci_low"] = coef_df["coef"] - 1.96 * coef_df["se"]
coef_df["ci_high"] = coef_df["coef"] + 1.96 * coef_df["se"]

In [108]:
# --- Split data into two panels ---
panel_a = coef_df[coef_df["model"] == "Triple Diff (by Age)"]
panel_b = coef_df[coef_df["model"].str.contains("DiD")]

# Determine common x-axis limits with padding
xmin = min(min(coef_df["ci_low"]) * 1.15, -0.01)  # Extend to include -0.01
xmax = max(coef_df["ci_high"]) * 1.15

# Create figure
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Panel A
y_positions_a = list(range(len(panel_a)))
axes[0].errorbar(
    panel_a["coef"], y_positions_a,
    xerr=[panel_a["coef"] - panel_a["ci_low"], panel_a["ci_high"] - panel_a["coef"]],
    fmt="none", color="gray", capsize=5, zorder=1
)
sns.scatterplot(
    data=panel_a, x="coef", y=y_positions_a,
    hue="group", palette="Set2", s=80, ax=axes[0], legend=False, zorder=3
)

# Add coefficient values above points
for i, (idx, row) in enumerate(panel_a.iterrows()):
    axes[0].text(row["coef"], i + 0.15, f'{row["coef"]:.3f}',
                 ha="center", va="center", color="black", fontsize=10)

axes[0].axvline(0, color="black", linestyle="--", zorder=0)
axes[0].set_title("Panel A. Triple Diff by Low-Income Vehicle Age Group", fontsize=14, pad=15)
axes[0].set_xlabel("Coefficient Estimate", fontsize=12)
axes[0].set_ylabel("Vehicle Age Group", fontsize=12)
axes[0].set_xlim(xmin, xmax)
axes[0].set_yticks(y_positions_a)
axes[0].set_yticklabels(panel_a["group"].tolist())
axes[0].set_ylim(-0.5, len(y_positions_a) - 0.5)

# Panel B
y_positions_b = list(range(len(panel_b)))
axes[1].errorbar(
    panel_b["coef"], y_positions_b,
    xerr=[panel_b["coef"] - panel_b["ci_low"], panel_b["ci_high"] - panel_b["coef"]],
    fmt="none", color="gray", capsize=5, zorder=1
)
sns.scatterplot(
    data=panel_b, x="coef", y=y_positions_b,
    hue="model", palette="Set1", s=80, ax=axes[1], legend=False, zorder=3  # Legend removed
)

# Add coefficient values above points
for i, (idx, row) in enumerate(panel_b.iterrows()):
    axes[1].text(row["coef"], i + 0.15, f'{row["coef"]:.3f}',
                 ha="center", va="center", color="black", fontsize=10)

axes[1].axvline(0, color="black", linestyle="--", zorder=0)
axes[1].set_title("Panel B. DiD by Poverty Tow Status", fontsize=14, pad=15)
axes[1].set_xlabel("Coefficient Estimate", fontsize=12)
axes[1].set_ylabel("")
axes[1].set_xlim(xmin, xmax)
axes[1].set_yticks(y_positions_b)
axes[1].set_yticklabels(panel_b["group"].tolist())
axes[1].set_ylim(-0.5, len(y_positions_b) - 0.5)

# Shared formatting
for ax in axes:
    ax.grid(True, linestyle="--", alpha=0.3)

plt.tight_layout()

plt.savefig("figure_2.png", dpi=300, bbox_inches="tight")

plt.show()